**Librerias a usar en el proceso de ETL**

In [43]:
import os
import pandas as pd
import json
import numpy as np
import pickle
import ast
pd.set_option('display.max_columns', None)

## 1- ETL de Archivos YELP

### 1.1- Archivo checkin.json

**Abrimos el archivo con la siguiente función:**

In [3]:
#Funcion para abrir chekin de yelp

def abrir_Archivo_json(archivo):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    with open(archivo) as file:
        for line in file:
            try:
                obj = json.loads(line)
                merged_data.append(obj)
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en el archivo {archivo}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

# Ejemplo de uso
archivo = '../checkin.json'
df_checkin_yelp = abrir_Archivo_json(archivo)

**Revisamos si tiene nulos**

In [4]:
df_checkin_yelp.isnull().sum()

business_id    0
date           0
dtype: int64

**Revisamos si tiene duplicados**

In [5]:
df_checkin_yelp.duplicated().sum()

0

**Desanidamos la fecha (en Fecha y hora)**

In [27]:
desanidados = []

for _, row in df_checkin_yelp.iterrows():
    business_id = row['business_id']
    dates = row['date'].split(', ')
    for date in dates:
        time, date = date.split(' ')
        desanidados.append([business_id, time, date])

df_desanidado = pd.DataFrame(desanidados, columns=['business_id', 'hour', 'fecha'])

**Finalmente se exporta**

In [ ]:
df_desanidado.to_csv('checkin.csv', index=False)

### 1.2- Archivo user.parquet

**Leemos el archivo**

In [24]:
archivo = "../user.parquet"
df_User_yelp = pd.read_parquet(archivo)

**Eliminamos duplicados y reindexamos**

In [25]:
df = df_User_yelp.drop_duplicates(subset=["user_id", "name"], keep="first")
df_User_yelp = df_User_yelp.reindex(df.index)
df_User_yelp['id_user'] = df.index

**Creamos y exportamos un nuevo dataframe para conservar id_user original con su user_id , el cual será usado para unir tablas**

In [26]:
dfusuario = df_User_yelp.loc[:, ["id_user",'user_id']]

In [27]:
#Renombramos la columna user_id

dfusuario.rename(columns={"user_id": "yelp_id"}, inplace=True)

In [ ]:
#Exportamos el nuevo dataframe llamado dfusuario
dfusuario.to_csv("UsuarioYelp")

**Eliminamos la columna user_id del dataframe df_user_yelp**

In [28]:
df_User_yelp.drop(columns=['user_id'], inplace=True)

**Aplicamos transformaciones a los tipos de datos en las diferentes variables**

In [32]:
#Cambiamos a string la variable name

df_User_yelp['name'] = df['name'].astype(str)

#cambiamos a tipo datetime la variable yelping_since
df_User_yelp['yelping_since'] = pd.to_datetime(df['yelping_since'])

#Creamos una columna año a partir de elite que devuelva una lista de años separado por comas
df_User_yelp['years'] = df['elite'].str.split(',')

#creamos un nuevo DataFrame con filas individuales para cada año y usuario
years_df = df_User_yelp[['id_user', 'years']].explode('years')

#reemplazamos valores vacios por np.nan en years
years_df['years'] = years_df['years'].replace("", np.nan)

#reemplazamos 20 po 2020 en years
years_df['years'] = years_df['years'].replace("20", "2020")

**Exportamos segundo dataframe de user_yelp como usuarioElite**

In [ ]:
years_df.to_csv("Datasets/UsuarioElite")

**Eliminamos las columnas elite y years del dataframe original user.parquet**

In [ ]:
df_User_yelp.drop(columns=['elite'], inplace=True)
df_User_yelp.drop(columns=['years'], inplace=True)

**Creamos una nueva columna llamada amigos a partir de friends**

In [34]:
df_User_yelp['amigos'] = df['friends'].str.split(',')

In [36]:
#creamos un nuevo DataFrame para desanidar amigos en filas individuales
dfamigos = df_User_yelp[['id_user', 'amigos']].explode('amigos')

In [37]:
#reemplazamos valores nulos por np.nan
dfamigos["amigos"] = dfamigos["amigos"].replace("None", np.nan)

**Exportamos dataframe amigos**

In [ ]:
dfamigos.to_csv("Datasets/friends")

## 1.3 Archivo business.pkl

**Leemos el archivo**

In [44]:
df_business= pd.read_pickle('../business.pkl')

In [46]:
df_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,IN,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,DE,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,AB,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,AB,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
